In [263]:
import ctypes
from pathlib import Path
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [264]:
class MapsForgeHeader(ctypes.BigEndianStructure):
    _pack_ = 1
    _fields_ = [
        ("magic_byte", ctypes.c_char * 20),
        ("header_size", ctypes.c_uint32),
        ("file_version", ctypes.c_uint32),
        ("file_size", ctypes.c_uint64),
        ("date_of_creation", ctypes.c_uint64),
        ("minLat", ctypes.c_int32),
        ("minLon", ctypes.c_int32),
        ("maxLat", ctypes.c_int32),  # most nothern point
        ("maxLon", ctypes.c_int32),
        ("tile_size", ctypes.c_uint16),
    ]


class ParsedFileName:
    country: str
    state: str
    creationDate: str
    minLong: int
    minLat: int
    sizeNorthSouth: int
    sizeWestEast: int

    def __init__(self, filename):
        self.country = filename[:2]
        self.state = filename[2:6]
        self.creationDate = filename[6:12]
        self.field1 = int(filename[12:15], 36)
        self.field2 = int(filename[15:18], 36)
        self.z1 = int(filename[18:21], 36)
        self.z2 = int(filename[21:24], 36)


def parseHeader(file: Path) -> MapsForgeHeader:
    header = MapsForgeHeader.from_buffer_copy(file.read_bytes())
    return header


def parseFilename(filename) -> ParsedFileName:
    return ParsedFileName(filename)

In [265]:
files = [f for f in Path("./allmaps").glob(("*.map")) if f.is_file()]
data = []


print("--- files ----   (", len(files), ")")

for i in files:
    print(i.name)

--- files ----   ( 727 )
asia_Bangladesh-BD00002206254PE2OA02Z05J.map
asia_Armenia-AM00002206253X82D2021021.map
asia_Kuwait-KW00002303103Z62LS01L017.map
asia_Iran-IR00002206253XL2EB0C80BT.map
asia_Turkmenistan-TM000022062542J2BQ09E06B.map
asia_Tajikistan-TJ00002206254CB2D904Z03L.map
asia_Afghanistan-AF00002311114802FD09406Z.map
asia_Bahrain-BH000023031041J2O900G00U.map
asia_Brunei-BN000023031055V32J00U00Q.map
asia_Cambodia-KH00002303104YG2WD03D03G.map
asia_China_Beijing-CN020025040255B2D902R04Q.map
asia_China_Anhui-CN010025040256C2ID039045.map
asia_China_Chongqing-CN030025040250B2K903N02Y.map
asia_China_Fujian-CN040025040256Y2MZ03503H.map
asia_China_Gansu-CN05002504024RV2BA0AV08Q.map
asia_China_Guangdong-CN06002504025332OT04U041.map
asia_China_Hainan-CN090025040252F2SP02M02Z.map
asia_China_Hebei-CN100025040255B2BT049066.map
asia_China_Guizhou-CN08002504024Z82MD03V03F.map
asia_China_Hongkong-CN130025040255Q2R400E00A.map
asia_China_Henan-CN12002504025352EJ04J06C.map
asia_China_Guangxi-CN

In [266]:
for f in files:
    header = parseHeader(f)
    # fix modified name (my maps are named like Vorarlberg-AT08002303103BS27H00H00P.map)
    humanName, filename = f.name.split("-")
    pf = parseFilename(filename)
    data.append({"file": f, "header": header, "parsedFilename": pf, "Name": humanName})

In [267]:
names = [d["Name"] for d in data]
minLat = [d["header"].minLat for d in data]
minLon = [d["header"].minLon for d in data]
maxLat = [d["header"].maxLat for d in data]
maxLon = [d["header"].maxLon for d in data]

f1 = [d["parsedFilename"].field1 for d in data]
f2 = [d["parsedFilename"].field2 for d in data]
f3 = [d["parsedFilename"].z1 for d in data]
f4 = [d["parsedFilename"].z2 for d in data]


df = pd.DataFrame(
    {
        "name": names,
        "minLat": minLat,
        "minLon": minLon,
        "maxLat": maxLat,
        "maxLon": maxLon,
        "f1": f1,
        "f2": f2,
        "f3": f3,
        "f4": f4,
    }
)
df["minLat"] = df["minLat"] / 1000000
df["minLon"] = df["minLon"] / 1000000
df["maxLat"] = df["maxLat"] / 1000000
df["maxLon"] = df["maxLon"] / 1000000

df["angleDiffLongitude"] = df["maxLon"] - df["minLon"]
df["angleDiffLatgitude"] = df["maxLat"] - df["minLat"]

df['continent'] = df['name'].str.split('_',n=1, expand=True)[0]

display(df)

,name,minLat,minLon,maxLat,maxLon,f1,f2,f3,f4,angleDiffLongitude,angleDiffLatgitude,continent
0,asia_Bangladesh,18.584030,87.99765,26.646770,92.69714,6098,3466,107,199,4.69949,8.062740,asia
1,asia_Armenia,38.837430,43.43821,41.310290,46.64262,5084,3062,73,73,3.20441,2.472860,asia
2,asia_Kuwait,28.495000,46.53500,30.125000,49.02500,5154,3376,57,43,2.49000,1.630000,asia
3,asia_Iran,24.039470,44.02303,39.790450,63.35413,5097,3107,440,425,19.33110,15.750980,asia
4,asia_Turkmenistan,35.123550,51.83271,42.860810,66.70892,5275,3014,338,227,14.87621,7.737260,asia
...,...,...,...,...,...,...,...,...,...,...,...,...
722,southamerica_Polivia,-22.873560,-69.64076,-9.680567,-57.45810,2511,4317,277,313,12.18266,13.192993,southamerica
723,southamerica_Suriname,1.810000,-58.09000,6.245000,-53.82500,2774,3953,97,101,4.26500,4.435000,southamerica
724,southamerica_TrinidadandTobago,9.855000,-62.10500,11.585000,-60.27000,2682,3830,42,40,1.83500,1.730000,southamerica
725,southamerica_Venezuela,0.621727,-73.38098,15.961340,-59.51474,2426,3727,315,354,13.86624,15.339613,southamerica


In [268]:
fig = px.scatter(
    df,
    x="maxLon",
    y="f1",
    color="continent",
    title="Value marked with <> dependend on Longitude AT0000111111 <3BR> 262 04W 02L in Base36",
    hover_name="name",
    hover_data=["maxLon", "f1"],
    labels={
        "f1": "Value from filename",
        "maxLon": "Longitude in Degrees",
    },
)
fig.write_html("docs/f1-maxLon.html")
fig.show()

In [269]:
fig = px.scatter(
    df,
    x="maxLat",
    y="f2",
    color="continent",
    title="Value marked with <> dependend on Latitude AT0000111111 3BR <262> 04W 02L in Base36",
    hover_name="name",
    hover_data=["maxLat", "f2"],
    labels={
        "f2": "Value from filename",
        "maxLat": "Latitude in Degrees",
    },
)
fig.write_html("docs/f2-maxLat.html")
fig.show()

In [270]:
fig = px.scatter(
    df,
    x="angleDiffLongitude",
    y="f3",
    color="continent",
    title="Value marked with <> dependend on Longitude difference AT0000111111 3BR 262 <04W> 02L in Base36",
    hover_name="name",
    hover_data=["angleDiffLongitude", "f3"],
    labels={
        "f3": "Value from filename",
        "angleDiffLongitude": "Longitude in Degrees",
    },
)
fig.write_html("docs/f3-angleDiffLongitude.html")
fig.show()

In [271]:
fig = px.scatter(
    df,
    x="angleDiffLatgitude",
    y="f4",
    color="continent",
    title="Value marked with <> dependend on Latitude difference AT0000111111 3BR 262 04W <02L> in Base36",
    hover_name="name",
    hover_data=["angleDiffLatgitude", "f4"],
    labels={
        "f4": "Value from filename",
        "angleDiffLatgitude": "Latitude in Degrees",
    },
)
fig.write_html("docs/f4-angleDiffLatgitude.html")
fig.show()

In [ ]:
long = []
lati = []
names = []

df = df.reset_index()

for index, row in df.iterrows():

    long.append(row["maxLon"])
    long.append(row["minLon"])
    long.append(row["minLon"])
    long.append(row["maxLon"])

    lati.append(row["maxLat"])
    lati.append(row["maxLat"])
    lati.append(row["minLat"])
    lati.append(row["minLat"])

    names.extend([row["name"]]*4)

    long.append(None)
    lati.append(None)
    names.append(None)

fig = go.Figure(go.Scattermap(mode="lines", fill="toself", lon=long, lat=lati, text=names))

fig.update_layout(
    map={"style": "carto-darkmatter", "center": {"lon": 30, "lat": 30}, "zoom": 2},
    showlegend=True,
    margin={"l": 0, "r": 0, "b": 0, "t": 0},
)

fig.write_html("docs/BoundingBoxes.html")
fig.show()

['asia_Bangladesh', 'asia_Bangladesh', 'asia_Bangladesh', 'asia_Bangladesh', None, 'asia_Armenia', 'asia_Armenia', 'asia_Armenia', 'asia_Armenia', None, 'asia_Kuwait', 'asia_Kuwait', 'asia_Kuwait', 'asia_Kuwait', None, 'asia_Iran', 'asia_Iran', 'asia_Iran', 'asia_Iran', None, 'asia_Turkmenistan', 'asia_Turkmenistan', 'asia_Turkmenistan', 'asia_Turkmenistan', None, 'asia_Tajikistan', 'asia_Tajikistan', 'asia_Tajikistan', 'asia_Tajikistan', None, 'asia_Afghanistan', 'asia_Afghanistan', 'asia_Afghanistan', 'asia_Afghanistan', None, 'asia_Bahrain', 'asia_Bahrain', 'asia_Bahrain', 'asia_Bahrain', None, 'asia_Brunei', 'asia_Brunei', 'asia_Brunei', 'asia_Brunei', None, 'asia_Cambodia', 'asia_Cambodia', 'asia_Cambodia', 'asia_Cambodia', None, 'asia_China_Beijing', 'asia_China_Beijing', 'asia_China_Beijing', 'asia_China_Beijing', None, 'asia_China_Anhui', 'asia_China_Anhui', 'asia_China_Anhui', 'asia_China_Anhui', None, 'asia_China_Chongqing', 'asia_China_Chongqing', 'asia_China_Chongqing', 'as